In [10]:
# ライブラリをインポート

# データ操作と数値計算のため
import pandas as pd
import numpy as np

# データ可視化のため
import matplotlib.pyplot as plt
import seaborn as sns

# 統計モデリングと計量経済分析のため
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit, Logit
from statsmodels.iolib.summary2 import summary_col

# for ignore warning
import warnings
warnings.filterwarnings('ignore')

In [3]:
url = "https://www.fbc.keio.ac.jp/~tyabu/keiryo/participation_data.csv"
df = pd.read_csv(url)
df.head()

,male,age,partner,children,yongest,experience,undergrad,grad,job
0,1,48,1,1,10,26,1,0,1
1,1,47,1,1,5,25,1,0,1
2,1,58,1,2,17,27,1,0,1
3,0,29,1,2,1,3,0,0,3
4,1,60,1,3,24,38,1,0,1


## 変数の定義

In [4]:
df['full'] = (df['job'] == 1).astype(int)
df['part'] = (df['job'] == 2).astype(int)
df['work'] = df['full'] + df['part']
df['under6'] = (df['yongest'] < 6).astype(int)
print(df.head())

   male  age  partner  children  yongest  experience  undergrad  grad  job  \
0     1   48        1         1       10          26          1     0    1   
1     1   47        1         1        5          25          1     0    1   
2     1   58        1         2       17          27          1     0    1   
3     0   29        1         2        1           3          0     0    3   
4     1   60        1         3       24          38          1     0    1   

   full  part  work  under6  
0     1     0     1       0  
1     1     0     1       1  
2     1     0     1       0  
3     0     0     0       1  
4     1     0     1       0  


## 12.6節の推定結果(50歳以下の男性に限定した分析)

In [6]:
df_male = df[(df['male'] == 1) & (df['age'] <= 50)]

### 線形確率

In [12]:
X = sm.add_constant(df_male[['experience', 'undergrad', 'grad', 'partner', 'under6', 'children']])
y = df_male['work']
male_lm = sm.OLS(y, X).fit(cov_type='HC1')
print(male_lm.summary())

                            OLS Regression Results                            
Dep. Variable:                   work   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     1.304
Date:                Fri, 30 Aug 2024   Prob (F-statistic):              0.253
Time:                        16:35:20   Log-Likelihood:                 458.01
No. Observations:                 682   AIC:                            -902.0
Df Residuals:                     675   BIC:                            -870.3
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8349      0.073     11.511      0.0

### プロビットモデル

In [13]:
male_probit = Probit(y, X).fit(cov_type='HC1')
print(male_probit.summary())

Optimization terminated successfully.
         Current function value: 0.070766
         Iterations 9
                          Probit Regression Results                           
Dep. Variable:                   work   No. Observations:                  682
Model:                         Probit   Df Residuals:                      675
Method:                           MLE   Df Model:                            6
Date:                Fri, 30 Aug 2024   Pseudo R-squ.:                  0.1429
Time:                        16:35:39   Log-Likelihood:                -48.262
converged:                       True   LL-Null:                       -56.309
Covariance Type:                  HC1   LLR p-value:                   0.01326
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0083      0.531     -0.016      0.988      -1.050       1.033
experience     0.0500      0.

### プロビットモデルの限界効果

In [14]:
male_probit_margins = male_probit.get_margeff()
print(male_probit_margins.summary())

       Probit Marginal Effects       
Dep. Variable:                   work
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.0018      0.001      1.873      0.061   -8.19e-05       0.004
undergrad      0.0239      0.014      1.664      0.096      -0.004       0.052
grad           0.0018      0.017      0.106      0.915      -0.031       0.034
partner        0.0261      0.013      1.983      0.047       0.000       0.052
under6         0.0180      0.013      1.380      0.168      -0.008       0.044
children       0.0042      0.007      0.607      0.544      -0.009       0.018


### ロジットモデル

In [15]:
male_logit = Logit(y, X).fit(cov_type='HC0')
print(male_logit.summary())

Optimization terminated successfully.
         Current function value: 0.070411
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                   work   No. Observations:                  682
Model:                          Logit   Df Residuals:                      675
Method:                           MLE   Df Model:                            6
Date:                Fri, 30 Aug 2024   Pseudo R-squ.:                  0.1472
Time:                        16:36:05   Log-Likelihood:                -48.020
converged:                       True   LL-Null:                       -56.309
Covariance Type:                  HC0   LLR p-value:                   0.01097
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7022      1.117     -0.628      0.530      -2.892       1.488
experience     0.1272      0

### ロジットモデルの限界効果

In [16]:
male_logit_margins = male_logit.get_margeff()
print(male_logit_margins.summary())

        Logit Marginal Effects       
Dep. Variable:                   work
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.0019      0.001      1.798      0.072      -0.000       0.004
undergrad      0.0256      0.017      1.477      0.140      -0.008       0.059
grad           0.0028      0.017      0.162      0.871      -0.031       0.036
partner        0.0241      0.012      2.029      0.042       0.001       0.047
under6         0.0181      0.014      1.339      0.181      -0.008       0.045
children       0.0020      0.007      0.288      0.774      -0.012       0.016


### 12.6節の推定結果(50歳以下の女性に限定した分析

In [18]:
df_female = df[(df['male'] == 0) & (df['age'] <= 50)]

### 線形確率

In [19]:
X = sm.add_constant(df_female[['experience', 'undergrad', 'grad', 'partner', 'under6', 'children']])
y = df_female['work']
female_lm = sm.OLS(y, X).fit(cov_type='HC1')
print(female_lm.summary())

                            OLS Regression Results                            
Dep. Variable:                   work   R-squared:                       0.201
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     49.75
Date:                Fri, 30 Aug 2024   Prob (F-statistic):           2.07e-53
Time:                        16:36:38   Log-Likelihood:                -558.63
No. Observations:                 976   AIC:                             1131.
Df Residuals:                     969   BIC:                             1165.
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4893      0.062      7.924      0.0

### プロビットモデル

In [20]:
female_probit = Probit(y, X).fit(cov_type='HC0')
print(female_probit.summary())

Optimization terminated successfully.
         Current function value: 0.539753
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                   work   No. Observations:                  976
Model:                         Probit   Df Residuals:                      969
Method:                           MLE   Df Model:                            6
Date:                Fri, 30 Aug 2024   Pseudo R-squ.:                  0.1737
Time:                        16:36:52   Log-Likelihood:                -526.80
converged:                       True   LL-Null:                       -637.53
Covariance Type:                  HC0   LLR p-value:                 5.083e-45
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1062      0.217     -0.489      0.625      -0.532       0.320
experience     0.0729      0.

### プロビットモデルの限界効果

In [21]:
female_probit_margins = female_probit.get_margeff()
print(female_probit_margins.summary())

       Probit Marginal Effects       
Dep. Variable:                   work
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.0223      0.002     10.613      0.000       0.018       0.026
undergrad      0.0264      0.036      0.738      0.460      -0.044       0.096
grad          -0.0235      0.131     -0.180      0.857      -0.280       0.233
partner       -0.1384      0.049     -2.822      0.005      -0.235      -0.042
under6        -0.1741      0.027     -6.422      0.000      -0.227      -0.121
children       0.0333      0.018      1.897      0.058      -0.001       0.068


### ロジットモデル

In [22]:
female_logit = Logit(y, X).fit(cov_type='HC0')
print(female_logit.summary())

Optimization terminated successfully.
         Current function value: 0.539305
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                   work   No. Observations:                  976
Model:                          Logit   Df Residuals:                      969
Method:                           MLE   Df Model:                            6
Date:                Fri, 30 Aug 2024   Pseudo R-squ.:                  0.1744
Time:                        16:37:10   Log-Likelihood:                -526.36
converged:                       True   LL-Null:                       -637.53
Covariance Type:                  HC0   LLR p-value:                 3.307e-45
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1605      0.373     -0.430      0.667      -0.892       0.571
experience     0.1255      0.

### ロジットモデルの限界効果

In [23]:
female_logit_margins = female_logit.get_margeff()
print(female_logit_margins.summary())

        Logit Marginal Effects       
Dep. Variable:                   work
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
experience     0.0228      0.002     10.713      0.000       0.019       0.027
undergrad      0.0276      0.035      0.783      0.434      -0.041       0.097
grad          -0.0224      0.128     -0.175      0.861      -0.274       0.229
partner       -0.1494      0.052     -2.866      0.004      -0.252      -0.047
under6        -0.1690      0.027     -6.366      0.000      -0.221      -0.117
children       0.0324      0.017      1.865      0.062      -0.002       0.067
